In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import plotly.express as px
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid

from tqdm import tqdm

# Data Loading

In [ ]:
# Load the Training Data
problem_title = "Bike count prediction"
_target_column_name = "log_bike_count"
# A type (class) which will be used to create wrapper objects for y_pred

def get_train_data(path="../input/mdsb-2023/train.parquet"):
    data = pd.read_parquet(path)
    # Sort by date first, so that time based cross-validation would produce correct results
    data = data.sort_values(["date", "counter_name"])
    y_array = data[_target_column_name].values
    X_df = data.drop([_target_column_name, "bike_count"], axis=1)
    return X_df, y_array
X, y = get_train_data()

In [ ]:
# We upload the scholar breaks data

link = 'https://raw.githubusercontent.com/AntoineAugusti/vacances-scolaires/master/data.csv'
data_school_holidays = pd.read_csv(link)
data_school_holidays['date'] = pd.to_datetime(data_school_holidays['date'])
data_school_holidays = data_school_holidays.dropna(axis=1)
keep_columns = ["date","vacances_zone_c"]
data_school_holidays = data_school_holidays[keep_columns]
data_school_holidays['vacances_zone_c'] = data_school_holidays['vacances_zone_c'].fillna(False)
data_school_holidays['vacances_zone_c'] = data_school_holidays['vacances_zone_c'].astype(int)
data_school_holidays

,date,vacances_zone_c
0,1990-01-01,0
1,1990-01-02,0
2,1990-01-03,0
3,1990-01-04,0
4,1990-01-05,0
...,...,...
13509,2026-12-27,0
13510,2026-12-28,0
13511,2026-12-29,0
13512,2026-12-30,0


# Data Splitting

In [ ]:
def train_test_split_temporal(X, y, delta_threshold="30 days"):

    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid
X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

# Model Training

In [ ]:
# updated version
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

class DateEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, holiday_dates):
        self.holiday_dates = holiday_dates

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['month'] = X['date'].dt.month
        X['day'] = X['date'].dt.day
        X['weekday'] = X['date'].dt.weekday
        X['hour'] = X['date'].dt.hour
        X['is_workday'] = np.where(X['weekday'] < 5, 1, 0)
        X['season'] = X['month'].apply(lambda x: (x%12 + 3)//3)

        X['day_sin'] = np.sin(2 * np.pi * X['weekday']/7)
        X['day_cos'] = np.cos(2 * np.pi * X['weekday']/7)
        X['month_sin'] = np.sin(2 * np.pi * X['month']/12)
        X['month_cos'] = np.cos(2 * np.pi * X['month']/12)

        X['vacances_zone_c'] = X['date'].isin(self.holiday_dates).astype(int)
        return X.drop(columns=["date"])

# Instantiate the DateEncoder with the holiday dates
# date_encoder = DateEncoder(holiday_dates)
date_encoder = DateEncoder(data_school_holidays[data_school_holidays['vacances_zone_c']==1 & data_school_holidays['date'].isin(X['date'])])


categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name"]

preprocessor = ColumnTransformer(
    [
        ("num", 'passthrough', ["month", "day", "weekday", "hour","season","day_sin","day_cos","month_sin","month_cos","is_workday","vacances_zone_c","latitude","longitude"]),
        ("cat", categorical_encoder, categorical_cols),
    ]
)

rg1 = LGBMRegressor(colsample_bytree=0.40927396991732046,
                    learning_rate=0.1956637125175451,
                    min_child_samples= 321,
                    min_child_weight=0.009676548190436696,
                    num_leaves= 118,
                    reg_alpha=0,
                    reg_lambda= 5,
                    subsample=0.4355591136556686
                   )

rg2 = XGBRegressor(colsample_bytree=0.6186133958679972,
                   learning_rate=0.04053474178612112,
                   max_depth=8,
                   min_child_weight=6,
                   n_estimators=602,
                   reg_alpha=0.4878098008087379,
                   reg_lambda=0.8508175178361462,
                   subsample=0.3615213371781007
                  )

rg3 = CatBoostRegressor(bagging_temperature=0.0906064345328208,
                        border_count=239,
                        depth=6,
                        iterations=975,
                        l2_leaf_reg=6.142344384136116
                       )

vrg = VotingRegressor(estimators=[('lgb', rg1), ('xgb', rg2), ('cgb', rg3)], weights=[1, 1, 1])

pipe = make_pipeline(date_encoder, preprocessor, vrg)

pipe.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027645 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 353
[LightGBM] [Info] Number of data points in the train set: 456507, number of used features: 98
[LightGBM] [Info] Start training from score 3.048868


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [17:45:32] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Learning rate set to 0.107739
0:	learn: 1.5663819	total: 90.2ms	remaining: 1m 30s
1:	learn: 1.4716275	total: 162ms	remaining: 1m 20s
2:	learn: 1.3907695	total: 228ms	remaining: 1m 15s
3:	learn: 1.3193635	total: 307ms	remaining: 1m 16s
4:	learn: 1.2604937	total: 375ms	remaining: 1m 14s
5:	learn: 1.2062422	total: 446ms	remaining: 1m 13s
6:	learn: 1.1601062	total: 515ms	remaining: 1m 13s
7:	learn: 1.1178988	total: 592ms	remaining: 1m 13s
8:	learn: 1.0828901	total: 666ms	remaining: 1m 13s
9:	learn: 1.0534759	total: 740ms	remaining: 1m 13s
10:	learn: 1.0212336	total: 809ms	remaining: 1m 12s
11:	learn: 0.9982455	total: 870ms	remaining: 1m 11s
12:	learn: 0.9744482	total: 957ms	remaining: 1m 12s
13:	learn: 0.9562397	total: 1.02s	remaining: 1m 12s
14:	learn: 0.9392072	total: 1.09s	remaining: 1m 11s
15:	learn: 0.9244521	total: 1.16s	remaining: 1m 11s
16:	learn: 0.9113742	total: 1.23s	remaining: 1m 11s
17:	learn: 0.8999832	total: 1.3s	remaining: 1m 10s
18:	learn: 0.8899219	total: 1.36s	remaining:

Pipeline(steps=[('dateencoder',
                 DateEncoder(holiday_dates=            date  vacances_zone_c
0     1990-01-01                0
1     1990-01-02                0
2     1990-01-03                0
3     1990-01-04                0
4     1990-01-05                0
...          ...              ...
13509 2026-12-27                0
13510 2026-12-28                0
13511 2026-12-29                0
13512 2026-12-30                0
13513 2026-12-31                0

[8965 rows x 2 columns])),
                ('columntransformer',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['month', 'day', 'weekday',
                                                   'hou...
                                                           learning_rate=0.003,
                                                           max_bin=2000,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=None,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=2500,
                                                           n_jobs=-1,
                                                           num_parallel_tree=None,
                                                           random_state=1, ...)),
                                             ('cgb',
                                              <catboost.core.CatBoostRegressor object at 0x7bc3cbeef250>)],
                                 weights=[2, 1, 2]))])

# Cross Validation

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# cv = TimeSeriesSplit(n_splits=6)

# # When using a scorer in scikit-learn it always needs to be better when smaller, hence the minus sign.
# scores = cross_val_score(
#     pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
# )
# print("RMSE: ", scores)
# print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

# Prediction

In [ ]:
final_test_path = '../input/mdsb-2023/final_test.parquet'
final_test = pd.read_parquet(final_test_path)

predicted_log_bike_count = pipe.predict(final_test)

predictions = pd.DataFrame({
    "Id": final_test.index,
    "log_bike_count": predicted_log_bike_count
})

predictions.to_csv('submission.csv', index=False)
